In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

from IPython.display import clear_output
from datetime import datetime

# = = = = = = = = = = = = = = =

import os

list_file = sorted(list(os.walk('./file/3.vehicle'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

# = = = = = = = = = = = = = = =

import shutil

if os.path.exists('./file/4.info'):
    shutil.rmtree('./file/4.info')

os.makedirs('./file/4.info')

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

import re

# = = = = = = = = = = = = = = =

import json

# = = = = = = = = = = = = = = =

for file in list_file:
    
    # = = = = = = = = = = = = = = =

    output_correct, output_error = pd.DataFrame(), pd.DataFrame()

    # = = = = = = = = = = = = = = =

    input_ = pd.read_excel(f'./file/3.vehicle/{file}',
                           header=0,
                           dtype=str).fillna('')
    
    input_['No'] = input_['No'].astype(int)

    print(f'总数量：{len(input_)}')
    print()

    # = = = = = = = = = = = = = = =


    work = Queue()
    for a in range(len(input_)):
        work.put_nowait(a)

    # = = = = = = = = = = = = = = =

    def crawler():
        global output_correct, output_error

        while not work.empty():
            a = work.get_nowait()

            # = = = = = = = = = = = = = = =

            crawler_status = 'error'

            # = = = = = = = = = = = = = = =

            payload = {'partData': {'listing_data_essential': {'parttype': input_.loc[a, 'Type Code'],
                                                               'partkey': input_.loc[a, 'Key']},
                                    'listing_data_supplemental': {'partnumber': input_.loc[a, 'Part Number'],
                                                                  'catalogname': input_.loc[a, 'Manufacturer']}}}
            
            data = {'func': 'getbuyersguide',
                    'payload': json.dumps(payload),
                    'api_json_request': '1'}

            try:
                b = 0
                while True:
                    b += 1

                    try:
                        resp = requests.get(input_.loc[a, 'Info'],
                                            headers=get_header(),
                                            proxies=get_proxy(),
                                            timeout=(10, 10))
                        
                        if resp.status_code == 200:
                            soup = BeautifulSoup(resp.text, 'lxml')
                            html = etree.HTML(str(soup))
                
                            if html.xpath('//span[@class="la-btn-v"]/text()')[0] == 'Continue Shopping':
                                break
                    except KeyboardInterrupt:
                        break
                    except:
                        continue

                # = = = = = = = = = = = = = = =

                dict_specification = {}
                list_tr = html.xpath('//table[@class="moreinfotable"]/tr')[1:] if html.xpath('//table[@class="moreinfotable"]/tr') else []
                for i, tr in zip(range(len(list_tr)), list_tr):
                    dict_specification[str(i)] = {}
                
                    list_td = [re.sub(r'<.*?>', '', etree.tostring(text).decode('utf-8')).strip() for text in tr.xpath('./td')]
                    for j, td in zip(range(len(list_td)), list_td):
                        dict_specification[str(i)][str(j)] = td

                # = = = = = = = = = = = = = = =

                dict_kit = {}
                list_tr = html.xpath('//section[contains(@aria-label, "Components") and contains(@aria-label, "kit")]/table/tbody/tr/td/table/tbody[contains(@class, "listing-inner") and (contains(@class, "altrow-b-0") or (contains(@class, "altrow-b-1")))]/tr')
                for i, tr in zip(range(len(list_tr)), list_tr):
                    dict_kit[str(i)] = {}
                
                    list_td = tr.xpath('./td')
                    dict_kit[str(i)]['quantity'] = int(list_td[0].xpath('./text()')[0].strip())
                
                    dict_kit[str(i)]['tag'] = list_td[1].xpath('./text()')[0].strip()
                
                    json_ = json.loads(list_td[2].xpath('./@value')[0])
                    dict_kit[str(i)]['type_code'] = json_['parttype']
                    dict_kit[str(i)]['key'] = json_['partkey']
                
                    json_2 = json.loads(tr.xpath('./input[contains(@id, "listing_data_supplemental[")]/@value')[0])
                    dict_kit[str(i)]['part_number'] = json_2['partnumber']
                    dict_kit[str(i)]['manufacturer'] = json_2['catalogname']
                
                    dict_kit[str(i)]['url'] = f'''https://www.rockauto.com/en/parts/{json_2['catalogname']},{json_2['partnumber']},{list_td[1].xpath('./text()')[0].strip()},{json_['parttype']}'''
                
                    list_info = tr.xpath('./div[@class="listing-text-row-moreinfo-truck"]/a[@class="ra-btn ra-btn-moreinfo"]/@href')
                    dict_kit[str(i)]['info'] = list_info[0].strip() if list_info and list_info[0].startswith('https://www.rockauto.com/en/moreinfo.php?') else f'''https://www.rockauto.com/en/moreinfo.php?pk={json_['partkey']}&pt={json_['parttype']}&Lennon=1'''
                
                    dict_src = {}
                    list_src = json.loads(list_td[3].xpath('./div/input[contains(@id, "jsninlineimg[")]/@value')[0])['Slots'] if list_td[3].xpath('./div/input[contains(@id, "jsninlineimg[")]/@value') else []
                    for j, src in zip(range(len(list_src)), list_src):
                        dict_src[str(j)] = 'https://www.rockauto.com' + src['ImageData']['Full'].strip()
                    dict_kit[str(i)]['dict_src'] = dict_src

                # = = = = = = = = = = = = = = =

                df_temp = pd.DataFrame([{'No': input_.loc[a, 'No'],
                                         'Manufacturer': input_.loc[a, 'Manufacturer'],
                                         'Group': input_.loc[a, 'Group'],
                                         'Type': input_.loc[a, 'Type'],
                                         'Type Code': input_.loc[a, 'Type Code'],
                                         'Part Number': input_.loc[a, 'Part Number'],
                                         'Parameter': input_.loc[a, 'Parameter'],
                                         'Json_Price': input_.loc[a, 'Json_Price'],
                                         'Note 1': input_.loc[a, 'Note 1'],
                                         'Note 2': input_.loc[a, 'Note 2'],
                                         'Json_Vehicle': input_.loc[a, 'Json_Vehicle'],
                                         'OEM': input_.loc[a, 'OEM'],
                                         'Picture': input_.loc[a, 'Picture'],
                                         'Url': input_.loc[a, 'Url'],
                                         'Info': input_.loc[a, 'Info'],
                                         'Json_Src': input_.loc[a, 'Json_Src'],
                                         'Json_Specification': json.dumps(dict_specification),
                                         'Json_Kit': json.dumps(dict_kit),
                                         'Key': input_.loc[a, 'Key']}])

                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

                # = = = = = = = = = = = = = = =

                crawler_status = 'ok'
            except:
                output_error = pd.concat([output_error, pd.DataFrame([input_.iloc[a]]).reset_index(drop=True)], ignore_index=True).fillna('')

            # = = = = = = = = = = = = = = =

            if list_file.index(file) % 3 == 0 and list_file.index(file) != 0 and work.qsize() == 0 or work.qsize() % 1_000 == 0 and work.qsize() != 0:
                clear_output()

            print(f'''[状态：{crawler_status}，尝试次数：{b}] - {(list_file.index(file) + 1) / len(list_file) * 100:.2f}% - {file.removesuffix('.xlsx')} > {input_.loc[a, 'Type']} > {input_.loc[a, 'Manufacturer']} > {input_.loc[a, 'No']}.{input_.loc[a, 'Part Number']}\n[剩余数量：{work.qsize()}] - [当前时间：{datetime.now().strftime('%H:%M:%S')}]\n''')

    # = = = = = = = = = = = = = = =

    list_task = []
    for _ in range(5):
        task = gevent.spawn(crawler)
        list_task.append(task)
    gevent.joinall(list_task)

    print('输出ing...')
    print()
    if not output_correct.empty:
        output_correct = output_correct.sort_values(by=['No'],
                                                    ascending=[True],
                                                    ignore_index=True)
        output_correct.to_excel(f'''./file/4.info/{file.removesuffix('.xlsx')}-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
    if not output_error.empty:
        output_error = output_error.sort_values(by=['No'],
                                                ascending=[True],
                                                ignore_index=True)
        output_error.to_excel(f'''./file/{file.removesuffix('.xlsx')}-info_error.xlsx''', index=False)
        print('爬虫存在error')
        print()

# = = = = = = = = = = = = = = =

list_file = sorted(list(os.walk('./file/4.info'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

# = = = = = = = = = = = = = = =

from tqdm import tqdm
import re

for file in tqdm(list_file, desc='Progress', ncols=77):
    os.rename(f'./file/4.info/{file}',
              f'''./file/4.info/{re.sub(r'-[0-9]{8}_[0-9]{6}.xlsx$', '.xlsx', file)}''')

print('Done ~')

总数量：507

[状态：ok，尝试次数：1] - 100.00% - crawler_1 > Timing Chain & Component Kit > MELLING > 23.31041S
[剩余数量：477] - [当前时间：15:20:12]

[状态：ok，尝试次数：1] - 100.00% - crawler_1 > Timing Chain & Component Kit > MELLING > 3.30420S
[剩余数量：476] - [当前时间：15:20:12]

[状态：ok，尝试次数：1] - 100.00% - crawler_1 > Timing Chain & Component Kit > MELLING > 17.31035S
[剩余数量：475] - [当前时间：15:20:12]

[状态：ok，尝试次数：1] - 100.00% - crawler_1 > Timing Chain & Component Kit > MELLING > 9.30470SX
[剩余数量：474] - [当前时间：15:20:12]

[状态：ok，尝试次数：1] - 100.00% - crawler_1 > Timing Chain & Component Kit > MELLING > 13.30490SX
[剩余数量：473] - [当前时间：15:20:12]

[状态：ok，尝试次数：1] - 100.00% - crawler_1 > Timing Chain & Component Kit > MELLING > 27.31047S
[剩余数量：472] - [当前时间：15:20:12]

[状态：ok，尝试次数：1] - 100.00% - crawler_1 > Timing Chain & Component Kit > MELLING > 2.30250SX
[剩余数量：471] - [当前时间：15:20:12]

[状态：ok，尝试次数：1] - 100.00% - crawler_1 > Timing Chain & Component Kit > MELLING > 14.30719S
[剩余数量：470] - [当前时间：15:20:12]

[状态：ok，尝试次数：1] - 100.00% - craw

Progress: 100%|██████████████████████████████| 1/1 [00:00<00:00, 2283.24it/s]

Done ~
